In [1]:
import numpy as np
import ujson
import bz2
import numpy as np
import pprint
import glob

In [2]:
DATA_DIR = '/disks/disk1/tingtingxuan/HFMLNewFiles-old/Signal'
data_dir = sorted(glob.glob(DATA_DIR + '/*.json'))

In [3]:
filename = data_dir[3]
event_index = 1

In [4]:
with open(filename,'rb') as z:
    raw_data = ujson.loads(z.read())

In [5]:
event = raw_data['Events'][event_index]

In [6]:
import sys
sys.path.append('..')

In [7]:
from parsers.parser_INTT_clustered import cluster_hits_by_event, pair

In [8]:
event_ID = event['MetaData']['EventID']
trigger = int(event['TruthTriggerFlag']['Flags']['D0toPiKInAcceptance'])
ip = np.array(event["MetaData"]['CollisionVertex'])
# the output event filename is consist of three part: 
# the first number represent whether this is a trgger event or not (1: trigger, 0: non-trigger)
# the second part is the self-defined file_number (to make they different)
# the thrid part is the eventID in json files. (0-999)
evtid = 'event%01i%05i%03i'%(trigger, 1, event_ID)

# precessing MVTX hits
n_MVTXHits = len(event['RawHit']['MVTXHits'])
n_INTTHits = len(event['RawHit']['INTTHITS'])
if n_MVTXHits == 0:
    empty_event += 1
    print('no MVTX hits here!')
elif n_INTTHits == 0:
    empty_event += 1
    print('no INTT hits here!')


pid_dic = {}
p_dic = {-1: {'OriginVertexPoint': None, 'TrackMomentum': None, 'TrackEnergy':None, 'ParticleTypeID':None, 'TriggerTrackFlag':None, 'is_complete_trk':None}}
pid_dic['MVTX'] = {}
pid_dic['INTT'] = {}
truth_tracks = event['TruthHit']['TruthTracks']

pos321 = []
neg321 = []
pos211 = []
neg211 = []
for track in truth_tracks:
    track['trigger_track_flag'] = False
    if track['ParticleTypeID'] == 321:
        pos321.append(track)
    elif track['ParticleTypeID'] == -321:
        neg321.append(track)
    elif track['ParticleTypeID'] == 211:
        pos211.append(track)
    elif track['ParticleTypeID'] == -211:
        neg211.append(track)
valid_trigger_flag = pair(pos321, neg211, ip) or pair(neg321, pos211, ip)

for truth_track in truth_tracks:
    is_complete_trk = True
    pid = truth_track['TrackSequenceInEvent']
    p_dic[pid] = {'OriginVertexPoint': truth_track['OriginVertexPoint'], 'TrackMomentum': truth_track['TrackMomentum'], 'TrackEnergy': truth_track['TrackEnergy'], 'ParticleTypeID': truth_track['ParticleTypeID'], 'TriggerTrackFlag': truth_track['trigger_track_flag']}
    for hit_id in truth_track['MVTXHitID'][0]:
        pid_dic['MVTX'][hit_id] = pid
    for hit_id in truth_track['INTTHitID'][0]:
        pid_dic['INTT'][hit_id + n_MVTXHits] = pid
    if len(truth_track['MVTXHitID'][0])==0 or len(truth_track['INTTHitID'][0])==0:
        is_complete_trk = False
    p_dic[pid]['is_complete_trk'] = is_complete_trk
hits_df = cluster_hits_by_event(event, pid_dic, p_dic)

In [9]:
np.array(sorted(pid_dic['MVTX'].keys()))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  29,  30,  31,  32,  33,  38,  39,  40,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  77,
        78,  79,  80,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 169, 170, 171, 172,
       173, 174, 175])

In [10]:
MVTXHits = event['RawHit']['MVTXHits']
INTTHits = event['RawHit']['INTTHITS']
tracks = event['TruthHit']['TruthTracks']

In [11]:
hits = []
track_id = []
for hit in MVTXHits:
    hits.append(hit['Coordinate'])
for hit in INTTHits:
    hits.append(hit['Coordinate'])
hits = np.array(hits)

mvtx_track_id = (-1) * np.ones(len(MVTXHits))
intt_track_id = (-1) * np.ones(len(INTTHits))
for track in tracks:
    # track['MVTXHitID'][0] = [i for i in track['MVTXHitID'][0] if i < len(MVTXHits)]
    # track['INTTHitID'][0] = [i for i in track['INTTHitID'][0] if i < len(INTTHits)]
    mvtx_track_id[track['MVTXHitID'][0]] = track['TrackID']
    intt_track_id[track['INTTHitID'][0]] = track['TrackID']

In [12]:
print(mvtx_track_id, intt_track_id)

[30. 30. 50. 50. 50. 50. 50. 54. 54. 54. 54. 54. 54. 54. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. 49. 49. 49. 22. 22. -1. -1.
 -1. -1.  4.  4.  4. -1. -1. -1. -1. -1. -1. -1. -1. -1. 31. 31. 30. 30.
 30. 50. 50. 50. 50. 54. 54. 54. 54. 54. 54. 54. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. 23. 23. 23. 23. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. 49. 49. 49. 22. 22. 22. 22. 22. 22.  4. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. 48. 48. 48. 31. 31. 31. 31. 54. 54. 54. 54. 54.
 54. -1. -1. -1. -1. -1. -1. -1. -1. -1. 30. 30. 30. 50. 50. 50. 50. 23.
 23. 23. 23. 49. 49. 22. 22. 22. 22. 22.  4.  4.  4.  4. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. 48. 48. 48. 31. 31. 31. 31.] [50. 23. 23.  4.  4. -1. -1. -1. -1. 48. 48. 31. 31. -1. -1. -1. -1. -1.
 -1. 30. 49. 22. -1. -1. -1. -1. -1. 30. 30. 50. 50.  4. -1. -1. 48. 54.
 54. 23. 23. 49. -1. -1. -1.]
